In [1]:
import pickle
from typing import Tuple

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import LSTM, Dense
from keras.models import Sequential, load_model
from keras.preprocessing.sequence import TimeseriesGenerator


import sys

sys.path.append("../src/")
from models.training_helpers import (
    get_timeseries_generator,
    get_features_and_target,
    get_multiple_input_timeseries_generator,
)

2024-01-08 23:52:13.532092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 23:52:13.532190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 23:52:13.535238: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 23:52:13.552166: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 23:52:14.675688: W tensorflow/compiler/tf2

In [2]:
TRAINING_SET = "../data/trusted/training_data.csv"
VALIDATION_SET = "../data/trusted/validation_data.csv"
# TEST_SET = '../data/trusted/test_set.csv'

MODEL_PATH = "../models_save/lstm"

In [3]:
X_train, y_train = get_features_and_target(
    pd.read_csv(TRAINING_SET, index_col=0)[:10000]
)
# X_val, y_val = get_features_and_target(pd.read_csv(VALIDATION_SET, index_col=0))

In [4]:
X_train.shape, y_train.shape

((10000, 61), (10000, 1))

In [5]:
X_train

,onpromotion,dcoilwtico,cluster,typedays_Additional,typedays_Event,typedays_Holiday,typedays_Normal,typesoldproducts_AUTOMOTIVE,typesoldproducts_BABY CARE,typesoldproducts_BEAUTY,...,state_Guayas,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua
0,0.0,0.792965,0.7500,False,False,True,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
1,0.0,0.792965,0.7500,False,False,True,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
2,0.0,0.792965,0.7500,False,False,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
3,0.0,0.792965,0.7500,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,0.0,0.792965,0.7500,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.793794,0.3125,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9996,0.0,0.793794,0.3125,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9997,0.0,0.793794,0.3125,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9998,0.0,0.793794,0.3125,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
training_generator = get_timeseries_generator(
    X_train, y_train, look_back=30, batch_size=1
)

In [7]:
# training_generator = (X, y) -> len = nb of lines in df - lookback = 9970
# X -> shape = (batch_size, look_back, n_features) = (1, 30, 61)
# y -> shape = (batch_size, n_features) = (1, 1)
len(training_generator)

9970

In [8]:
X_train

,onpromotion,dcoilwtico,cluster,typedays_Additional,typedays_Event,typedays_Holiday,typedays_Normal,typesoldproducts_AUTOMOTIVE,typesoldproducts_BABY CARE,typesoldproducts_BEAUTY,...,state_Guayas,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua
0,0.0,0.792965,0.7500,False,False,True,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
1,0.0,0.792965,0.7500,False,False,True,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
2,0.0,0.792965,0.7500,False,False,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
3,0.0,0.792965,0.7500,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,0.0,0.792965,0.7500,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.793794,0.3125,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9996,0.0,0.793794,0.3125,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9997,0.0,0.793794,0.3125,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9998,0.0,0.793794,0.3125,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
X_continuous = X_train[["onpromotion", "dcoilwtico", "cluster"]]
X_categorical = X_train.drop(["onpromotion", "dcoilwtico", "cluster"], axis=1)

In [12]:
X_continuous

,onpromotion,dcoilwtico,cluster
0,0.0,0.792965,0.7500
1,0.0,0.792965,0.7500
2,0.0,0.792965,0.7500
3,0.0,0.792965,0.7500
4,0.0,0.792965,0.7500
...,...,...,...
9995,0.0,0.793794,0.3125
9996,0.0,0.793794,0.3125
9997,0.0,0.793794,0.3125
9998,0.0,0.793794,0.3125


In [13]:
X_categorical

,typedays_Additional,typedays_Event,typedays_Holiday,typedays_Normal,typesoldproducts_AUTOMOTIVE,typesoldproducts_BABY CARE,typesoldproducts_BEAUTY,typesoldproducts_BEVERAGES,typesoldproducts_BOOKS,typesoldproducts_BREAD/BAKERY,...,state_Guayas,state_Imbabura,state_Loja,state_Los Rios,state_Manabi,state_Pastaza,state_Pichincha,state_Santa Elena,state_Santo Domingo de los Tsachilas,state_Tungurahua
0,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9996,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9997,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9998,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
training_multiple_generator = get_multiple_input_timeseries_generator(
    X_continuous, X_categorical, y_train, look_back=30, batch_size=1
)

TypeError: MultipleInputTimeseriesGenerator.__init__() got an unexpected keyword argument 'length'